In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\User\RR_model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001F7D7ED39B0>, origin='C:\\Users\\User\\RR_model\\src\\__init__.py', submodule_search_locations=['C:\\Users\\User\\RR_model\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob
#from src.rollrate.forecast import forecast_report
from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# # --- 2️⃣ Load dữ liệu thực tế ---
# sql_path = r"D:\Python_code\RR_Model\sql\NTB.sql"

# df = load_data(sql_path)
# print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")
df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].dt.replace(day=1)

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\User\RR_model\data\parquet
✅ Loaded 3,547,839 rows via pyarrow.dataset from C:\Users\User\RR_model\data\parquet
✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [4]:
import importlib
import src.rollrate.transition as transition

importlib.reload(transition)
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

In [ ]:
print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

count_fallback = 0
count_real = 0

for prod, prod_dict in matrices_by_mob.items():
    for mob, mob_dict in prod_dict.items():
        for score, obj in mob_dict.items():
            # obj là dict: {"P": ..., "is_fallback": bool, "reason": str}
            if obj.get("is_fallback", False):
                count_fallback += 1
            else:
                count_real += 1

print(f"\n🔹 Ma trận thực (real): {count_real}")
print(f"🔸 Ma trận fallback   : {count_fallback}")
print(f"🔢 Tổng               : {count_real + count_fallback}")


In [6]:
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.export_excel import export_transition_excel
from src.config import OUT_ROOT

print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

# Đếm real vs fallback
count_real = 0
count_fb = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            if obj["is_fallback"]:
                count_fb += 1
            else:
                count_real += 1

print(f"🔹 Ma trận REAL     : {count_real}")
print(f"🔸 Ma trận FALLBACK : {count_fb}")
print(f"🔢 Tổng             : {count_real + count_fb}")





=== Tính transition matrices theo Product × MOB × Score ===
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=B)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=C)
⚠️ Có 3 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=D)
⚠️ Có 7 hàng có tổng weight = 0: ['DPD30+', 'DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 7 hàng có tổng weight = 0: ['DPD60+', 'DPD90+', 'DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 6 hàng có tổng weight = 0: ['DPD90+', 'DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 5 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'WRITEOFF', 'SOLDOUT']
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 

In [ ]:


export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score.xlsx"
)


In [ ]:
# ================================================================
# NOTEBOOK TEST TRANSITION – Full Debug Version
# ================================================================

import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Import module transition chúng ta đã hoàn thiện
from src.rollrate.transition import (
    make_pairs,
    compute_transition_from_pairs,
    compute_transition_by_mob
)

from src.config import (
    CFG,
    BUCKETS_CANON,
    WEIGHT_METHOD,
    ROLL_WINDOW,
    MIN_OBS,
    MIN_EAD,
)

# =================================================================
# 1️⃣ CHỌN PRODUCT × SCORE × MOB ĐỂ TEST
# =================================================================

PRODUCT = "SALPIL"
SCORE = "A"
MOB = 0

print("=== 🧪 TEST TRANSITION FOR ===")
print(f"Product = {PRODUCT}, Score = {SCORE}, MOB = {MOB}")

# =================================================================
# 2️⃣ LOAD RAW DATA
# =================================================================

df_raw = df.copy()   # df là data gốc bạn đã load trước đó
print(f"📦 Raw data size: {df_raw.shape}")

# =================================================================
# 3️⃣ TẠO Pairs đầy đủ với WHA + EAD (thay thế EAD thành weight)
# =================================================================

pairs = make_pairs(df_raw)

print("\n=== 🔍 SỐ Pairs THEO PRODUCT × SCORE × MOB ===")
test_slice = pairs[
    (pairs["product_t"] == PRODUCT) &
    (pairs["score_t"] == SCORE) &
    (pairs["mob_t"] == MOB)
].copy()

print(f"Số cặp hợp lệ tìm được: {len(test_slice):,}")
display(test_slice.head(20))

if test_slice.empty:
    raise ValueError("❌ Không có cặp nào trong nhóm bạn chọn → không test được.")

# =================================================================
# 4️⃣ RAW COUNT transition (không ead, không wha)
# =================================================================

test_slice["raw_weight"] = 1

raw_mat = pd.crosstab(
    test_slice["state_t"],
    test_slice["state_t1"],
    values=test_slice["raw_weight"],
    aggfunc="sum"
).reindex(index=BUCKETS_CANON, columns=BUCKETS_CANON, fill_value=0)

raw_mat_norm = raw_mat.div(raw_mat.sum(axis=1).replace(0,np.nan), axis=0).fillna(0)

print("\n=== 🟦 RAW COUNT MATRIX (NO EAD – NO WHA) ===")
display(raw_mat_norm.style.format("{:.4f}"))

# =================================================================
# 5️⃣ EAD WEIGHT transition
# =================================================================

if CFG.get("ead") and CFG["ead"] in df_raw.columns:
    test_slice["ead_weight"] = test_slice["weight"] / (test_slice["weight"] / test_slice["weight"])  # = EAD ???
else:
    test_slice["ead_weight"] = 1

ead_mat = pd.crosstab(
    test_slice["state_t"],
    test_slice["state_t1"],
    values=test_slice["ead_weight"],
    aggfunc="sum"
).reindex(index=BUCKETS_CANON, columns=BUCKETS_CANON, fill_value=0)

ead_mat_norm = ead_mat.div(ead_mat.sum(axis=1).replace(0,np.nan), axis=0).fillna(0)

print("\n=== 🟧 EAD WEIGHTED MATRIX ===")
display(ead_mat_norm.style.format("{:.4f}"))

# =================================================================
# 6️⃣ EAD × WHA WEIGHT transition (trọng số thật sự)
# =================================================================

w_mat = pd.crosstab(
    test_slice["state_t"],
    test_slice["state_t1"],
    values=test_slice["weight"],
    aggfunc="sum"
).reindex(index=BUCKETS_CANON, columns=BUCKETS_CANON, fill_value=0)

w_mat_norm = w_mat.div(w_mat.sum(axis=1).replace(0,np.nan), axis=0).fillna(0)

print("\n=== 🟩 EAD × WHA WEIGHTED MATRIX (CHÍNH XÁC) ===")
display(w_mat_norm.style.format("{:.4f}"))

# =================================================================
# 7️⃣ TÍNH BẰNG HÀM CHÍNH compute_transition_by_mob
# =================================================================

matrices, parents = compute_transition_by_mob(df_raw)

P_model = matrices[PRODUCT][MOB][SCORE]

print("\n=== 🟪 MATRIX FROM MODEL (compute_transition_by_mob) ===")
display(P_model.style.format("{:.4f}"))

# =================================================================
# 8️⃣ So sánh 3 ma trận
# =================================================================

compare_df = pd.DataFrame({
    "RAW→DPD0": raw_mat_norm.loc["DPD0"].values,
    "EAD→DPD0": ead_mat_norm.loc["DPD0"].values,
    "WHA→DPD0": w_mat_norm.loc["DPD0"].values,
    "MODEL→DPD0": P_model.loc["DPD0"].values,
}, index=BUCKETS_CANON)

print("\n=== 📊 COMPARE (Row = DPD0) ===")
display(compare_df.style.format("{:.4f}"))

print("\n🎯 DONE — Bạn có thể xem từng ma trận để biết sai lệch đến từ đâu.")


In [ ]:
# ============================================================
# 🔍 NOTEBOOK TEST TRANSITION – FULL DEBUG VERSION
# ============================================================

import pandas as pd
import numpy as np
from IPython.display import display

from src.rollrate.transition import (
    make_pairs,
    compute_transition_from_pairs,
    compute_transition_by_mob,
)
from src.config import CFG, BUCKETS_CANON, SEGMENT_COLS


# ============================================================
# 1️⃣ STEP 1 – CHECK RAW DATA
# ============================================================
print("=== CHECK RAW DATA ===")

needed_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["cutoff"]]
for col in needed_cols:
    if col not in df.columns:
        print(f"❌ Thiếu cột bắt buộc: {col}")
    else:
        print(f"✔️ Có cột: {col}")

print("\n📌 SAMPLE RAW DATA")
display(df.head())

print("\n🔢 Unique MOB:", sorted(pd.to_numeric(df[CFG["mob"]], errors="coerce").dropna().unique()))
print("🔢 Unique STATE:", sorted(df[CFG["state"]].dropna().unique()))


# ============================================================
# 2️⃣ STEP 2 – CHECK PAIR GENERATION
# ============================================================
print("\n\n=== STEP 2 – MAKE PAIRS (MOB → MOB+1) ===")

pairs = make_pairs(df)

print("\n📌 Số cặp tạo được:", len(pairs))
print("\n📌 5 dòng đầu:")
display(pairs.head())

print("\n📌 Kiểm tra thiếu state ngoài BUCKETS_CANON:")
states_seen = set(pairs["state_t"]).union(set(pairs["state_t1"]))
missing = states_seen - set(BUCKETS_CANON)
print("⚠️ State ngoài BUCKETS_CANON:", missing)

print("\n📌 Thống kê pairs theo MOB:")
display(pairs.groupby("mob_t").size().rename("count").to_frame())

print("\n📌 Thống kê pairs theo (product,score):")
if "product_t" in pairs.columns:
    display(pairs.groupby(["product_t", "score_t"]).size().rename("count").to_frame())


# ============================================================
# 3️⃣ STEP 3 – DEBUG TRANSITION OF A SINGLE SLICE
# ============================================================

# Gợi ý debug với 1 nhóm (product, score, mob)
test_product = "SALPIL"
test_score   = "A"
test_mob     = 0

print(f"\n\n=== STEP 3 – DEBUG 1 SLICE: PRODUCT={test_product}, SCORE={test_score}, MOB={test_mob} ===")

slice_grp = pairs[
    (pairs["product_t"] == test_product)
    & (pairs["score_t"] == test_score)
    & (pairs["mob_t"] == test_mob)
]

print(f"📌 Số pairs trong slice: {len(slice_grp)}")
display(slice_grp.head())

print("\n📌 Tính thử transition matrix:")
P_test = compute_transition_from_pairs(slice_grp)
display(P_test)


# ============================================================
# 4️⃣ STEP 4 – DEBUG FULL compute_transition_by_mob
# ============================================================
print("\n\n=== STEP 4 – FULL TRANSITION BUILD (Product × MOB × Score) ===")

matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

print("\n📌 Danh sách product:")
print(list(matrices_by_mob.keys()))

print("\n📌 Danh sách fallback parent:")
display(parent_fallback.keys())


# ============================================================
# 5️⃣ STEP 5 – LIST ALL MATRICES WITH DEBUG FLAG
# ============================================================
print("\n\n=== STEP 5 – KIỂM TRA REAL vs FALLBACK MATRICES ===")

count_real = 0
count_fallback = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            if obj["is_fallback"]:
                count_fallback += 1
            else:
                count_real += 1

print(f"\n🔹 Ma trận REAL     : {count_real}")
print(f"🔸 Ma trận FALLBACK : {count_fallback}")
print(f"🔢 Tổng             : {count_real + count_fallback}")


# ============================================================
# 6️⃣ STEP 6 – DEBUG SPECIFIC MATRIX
# ============================================================

debug_prod = "SALPIL"
debug_score = "A"
debug_mob = 0

print(f"\n\n=== STEP 6 – SHOW SPECIFIC MATRIX (product={debug_prod}, score={debug_score}, mob={debug_mob}) ===")

try:
    obj = matrices_by_mob[debug_prod][debug_mob][debug_score]
    print("is_fallback:", obj["is_fallback"])
    print("reason:", obj["reason"])
    display(obj["P"])
except Exception as e:
    print("❌ Error:", e)


In [ ]:
# ============================================================
# 6️⃣ STEP 6 – DEBUG SPECIFIC MATRIX (FULL RAW CHECK)
# ============================================================

debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

print(f"\n=== DEBUG SPECIFIC MATRIX: PRODUCT={debug_prod}, SCORE={debug_score}, MOB={debug_mob} ===\n")

# ------------------------------------------------------------
# 1) Lấy raw dòng từ nguyên df trước khi làm pairs
# ------------------------------------------------------------
raw_slice = df[
    (df.get("PRODUCT_TYPE") == debug_prod) &
    (df.get("RISK_SCORE") == debug_score) &
    (df[CFG["mob"]].astype(int) == debug_mob)
].copy()

print("🔍 RAW DATA FOR THIS GROUP (before pairing)")
print(f"👉 Số dòng raw = {len(raw_slice):,}")
display(raw_slice.head(20))


# ------------------------------------------------------------
# 2) Lấy pairs từ make_pairs() cho slice này
# ------------------------------------------------------------
pairs_slice = pairs[
    (pairs["product_t"] == debug_prod) &
    (pairs["score_t"] == debug_score) &
    (pairs["mob_t"] == debug_mob)
].copy()

print("\n🔍 PAIRS FOR THIS GROUP (after make_pairs)")
print(f"👉 Số cặp MOB{debug_mob}→MOB{debug_mob+1} = {len(pairs_slice):,}")
display(pairs_slice.head(20))

# Summary theo state
print("\n📊 Count theo state_t (before weight):")
display(pairs_slice["state_t"].value_counts())

print("\n📊 Count theo state_t1 (before weight):")
display(pairs_slice["state_t1"].value_counts())


# ------------------------------------------------------------
# 3) Kiểm tra Weight (WHA weight × EAD)
# ------------------------------------------------------------
if "weight" in pairs_slice.columns:
    print("\n🔍 WEIGHT CHECK (WHA × EAD)")
    print("👉 Tổng weight =", pairs_slice["weight"].sum())
    print("👉 Thống kê weight:")
    display(pairs_slice["weight"].describe())
else:
    print("\n⚠️ pairs_slice không có cột 'weight' — có thể WHA chưa tính xong!")


# ------------------------------------------------------------
# 4) Manual aggregation để đối chiếu ma trận
# ------------------------------------------------------------
print("\n🔧 MANUAL AGGREGATE (state_t → state_t1)")

manual_ct = (
    pairs_slice
    .groupby(["state_t", "state_t1"])["weight"]
    .sum()
    .unstack(fill_value=0)
    .reindex(index=BUCKETS_CANON, columns=BUCKETS_CANON, fill_value=0)
)

display(manual_ct)


print("\n🔧 MANUAL NORMALIZED MATRIX (state_t → prob)")
manual_P = manual_ct.div(manual_ct.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
display(manual_P)


# ------------------------------------------------------------
# 5) Lấy ma trận thật từ compute_transition_by_mob
# ------------------------------------------------------------
print("\n🔍 FINAL MATRIX (from compute_transition_by_mob)")

try:
    obj = matrices_by_mob[debug_prod][debug_mob][debug_score]
    print("is_fallback:", obj["is_fallback"])
    print("reason:", obj["reason"])
    P_final = obj["P"]
    display(P_final)
except Exception as e:
    print("❌ Error when retrieving final matrix:", e)


# ------------------------------------------------------------
# 6) Compare manual vs final
# ------------------------------------------------------------
print("\n🔎 COMPARE manual_P vs P_final")

try:
    compare_df = pd.DataFrame({
        "manual": manual_P.stack(),
        "final" : P_final.stack(),
        "diff"  : (manual_P - P_final).stack()
    })
    display(compare_df[abs(compare_df["diff"]) > 0.00001])
except Exception as e:
    print("⚠️ Compare error:", e)


## Check data không đủ

In [ ]:
print("\n==============================================")
print("STEP 6 – FULL DEBUG: RAW → PAIRS → WEIGHTS → TRANSITION")
print("==============================================\n")

debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

loan_col  = CFG["loan"]
mob_col   = "mob_t"
state_col = "state_t"
cutoff    = "CUTOFF_DATE"
ead_col   = "ead_t"


# ------------------------------------------------------------
# 1) RAW DATA
# ------------------------------------------------------------
raw_slice = df[
    (df["PRODUCT_TYPE"] == debug_prod) &
    (df["RISK_SCORE"] == debug_score) &
    (df[CFG["mob"]].astype(int) == debug_mob)
].copy()

print(f"🔍 RAW DATA — PRODUCT={debug_prod}, SCORE={debug_score}, MOB={debug_mob}")
print(f"Số dòng raw   = {len(raw_slice):,}")
display(raw_slice.head())

print("\n📊 RAW State distribution:")
display(raw_slice[CFG["state"]].value_counts())


# ------------------------------------------------------------
# 2) PAIRS (t → t+1)
# ------------------------------------------------------------
pairs_slice = pairs_all[
    (pairs_all["product_t"] == debug_prod) &
    (pairs_all["score_t"] == debug_score) &
    (pairs_all["mob_t"] == debug_mob)
].copy()

print("\n🔍 PAIRS (t → t+1) của nhóm này")
print(f"Số pairs = {len(pairs_slice):,}")
display(pairs_slice.head())

print("\n📊 Pair count (state_t → state_t1):")
display(
    pairs_slice.groupby(["state_t", "state_t1"])
               .size()
               .unstack(fill_value=0)
)


# ------------------------------------------------------------
# 3) Xác định loans bị mất tháng kế tiếp
# ------------------------------------------------------------
loans_raw  = set(raw_slice[loan_col])
loans_pair = set(pairs_slice[loan_col])

missing = loans_raw - loans_pair
print(f"\n🚨 Loans thiếu mob_t+1 (không tạo được cặp): {len(missing)}")
display(list(missing)[:30])


# ------------------------------------------------------------
# 4) FULL LOAN TIMELINE
# ------------------------------------------------------------
print("\n📘 FULL LOAN TIMELINE — kiểm tra chuỗi MOB từng hợp đồng")
timeline = df[df[loan_col].isin(loans_raw)] \
    .sort_values([loan_col, CFG["mob"]])
display(timeline.head(50))


# ------------------------------------------------------------
# 5) TÍNH WEIGHT WHA CHÍNH XÁC (theo ROLL_WINDOW & WEIGHT_METHOD)
# ------------------------------------------------------------
from src.config import WEIGHT_METHOD, ROLL_WINDOW

ps = pairs_slice.copy()
ps[cutoff] = pd.to_datetime(ps["cutoff_t"], errors="coerce")

# Convert cutoff_date → monthly index
month_idx = ps[cutoff].dt.to_period("M").astype(int)
max_idx   = month_idx.max()

age = (max_idx - month_idx).clip(0, ROLL_WINDOW)

if WEIGHT_METHOD == "exp":
    lam = CFG.get("DECAY_LAMBDA", 0.9)
    ps["time_weight"] = (lam ** age).astype(float)
elif WEIGHT_METHOD == "linear":
    ps["time_weight"] = (1 - age / ROLL_WINDOW).clip(0)
else:
    ps["time_weight"] = 1.0

# final weight = EAD × time_weight
ps["weight"] = ps[ead_col] * ps["time_weight"]

print("\n🔍 WEIGHTED PAIRS (WHA × EAD)")
display(ps.head(20))

print("\n📊 Total Weight by state_t → state_t1:")
display(
    ps.groupby(["state_t", "state_t1"])["weight"]
      .sum()
      .unstack(fill_value=0)
)


# ------------------------------------------------------------
# 6) MANUAL TRANSITION MATRIX (INPUT CHUẨN)
# ------------------------------------------------------------
manual_counts = (
    ps.groupby(["state_t", "state_t1"])["weight"]
      .sum()
      .unstack(fill_value=0)
      .reindex(index=BUCKETS_CANON, columns=BUCKETS_CANON, fill_value=0)
)

manual_P = manual_counts.div(manual_counts.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

print("\n📊 MANUAL WEIGHTED TRANSITION MATRIX (đây là P thật trước khi smoothing):")
display(manual_P.round(4))


In [ ]:
loan_col = CFG["loan"]

state_summary = (
    seg_pairs
    .groupby("state_t")
    .agg(
        n_pairs=("state_t", "size"),             # số cặp transition đi ra từ state_t
        total_ead=("ead_t", "sum"),             # tổng EAD t tại state_t
        n_loans=(loan_col, "nunique")           # số hợp đồng khác nhau từng state_t
    )
    .reindex(BUCKETS_CANON, fill_value=0)       # đảm bảo đủ các bucket
)

print("🔎 Summary theo state_t (segment SALPIL-D):")
display(state_summary)


In [ ]:
print("\n=== CHECK TRÙNG / FALLBACK ===")
count_same = 0
count_total = 0

for prod, mob_dict in matrices_by_mob.items():
    for mob, mat in mob_dict.items():
        if isinstance(mat, dict):
            # So sánh score-level matrices trong cùng MOB
            mats = list(mat.values())
            base = mats[0].round(3)
            for m in mats[1:]:
                count_total += 1
                if m.round(3).equals(base):
                    count_same += 1

print(f"⚙️ {count_same}/{count_total} nhóm Score có ma trận giống hệt nhau (fallback).")


In [ ]:
# import pandas as pd
# import numpy as np
# from openpyxl import Workbook
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl.styles import Font, Alignment
# from pathlib import Path

# def export_transition_excel(matrices_by_mob, out_path):
#     """
#     Xuất ma trận chuyển trạng thái ra Excel:
#     - Mỗi PRODUCT_TYPE là 1 sheet.
#     - Trong sheet: sắp xếp theo Score tăng dần.
#       Mỗi MOB nằm ngang, cách nhau 2 cột trống.
#     """
#     wb = Workbook()
#     wb.remove(wb.active)
#     out_path = Path(out_path)

#     # Xác định các sản phẩm
#     for prod, mob_dict in matrices_by_mob.items():
#         ws = wb.create_sheet(str(prod)[:31])
#         row_cursor = 1  # dòng bắt đầu để viết

#         # Kiểm tra có score level hay không
#         has_score = any(isinstance(v, dict) for v in mob_dict.values())
#         if not has_score:
#             # --- Trường hợp chỉ có mob_dict[MOB] = matrix ---
#             mob_list = sorted(mob_dict.keys())
#             for mob in mob_list:
#                 mat = mob_dict[mob]
#                 header = f"{prod}_MOB{mob}"
#                 ws.cell(row=row_cursor, column=1, value=header).font = Font(bold=True)
#                 ws.cell(row=row_cursor, column=1).alignment = Alignment(horizontal="center")
#                 row_cursor += 1

#                 for r in dataframe_to_rows(mat.reset_index(), index=False, header=True):
#                     ws.append(r)
#                 row_cursor += mat.shape[0] + 3
#         else:
#             # --- Trường hợp có 3 tầng: MOB → Score → matrix ---
#             all_scores = sorted({s for mob in mob_dict.values() for s in mob.keys()})
#             all_mobs = sorted(mob_dict.keys())
#             base_row = 1

#             for score in all_scores:
#                 col_cursor = 1
#                 ws.cell(row=base_row, column=1, value=f"Score: {score}").font = Font(bold=True)
#                 for mob in all_mobs:
#                     if mob not in mob_dict:
#                         continue
#                     mat_dict = mob_dict[mob]
#                     if score not in mat_dict:
#                         continue
#                     mat = mat_dict[score]
#                     # Header cho block này
#                     header = f"{prod}_{score}_MOB{mob}"
#                     ws.cell(row=base_row + 1, column=col_cursor, value=header).font = Font(bold=True)
#                     ws.cell(row=base_row + 1, column=col_cursor).alignment = Alignment(horizontal="center")

#                     # Ghi ma trận vào Excel
#                     df_mat = mat.round(3).reset_index()
#                     for i, r in enumerate(dataframe_to_rows(df_mat, index=False, header=True)):
#                         for j, val in enumerate(r, start=0):
#                             ws.cell(row=base_row + 2 + i, column=col_cursor + j, value=val)
#                     # Di chuyển sang phải (ma trận width + 2 cột trống)
#                     col_cursor += mat.shape[1] + 3

#                 # Sau mỗi Score band, xuống dưới một block
#                 base_row += mat.shape[0] + 6

#     wb.save(out_path)
#     print(f"✅ Xuất ma trận Markov hoàn tất: {out_path}")


In [ ]:
export_transition_excel(
    matrices_by_mob,
    parent_fallback,
    out_path=OUT_ROOT / "Transition_ByProduct_MOB_Score.xlsx"
)


In [ ]:
# import pandas as pd
# import numpy as np
# from openpyxl import Workbook
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl.styles import Font, Alignment
# from pathlib import Path

# def export_transition_excel(matrices_by_mob, out_path, transpose=True):
#     """
#     Xuất ma trận chuyển trạng thái ra Excel:
#     - Mỗi PRODUCT_TYPE = 1 sheet duy nhất.
#     - Trong sheet: sắp theo Score tăng dần.
#       Nếu transpose=True → mỗi MOB xếp dọc (theo hàng, dễ xem nhiều MOB).
#       Nếu transpose=False → mỗi MOB xếp ngang (mặc định cũ).
#     """
#     wb = Workbook()
#     wb.remove(wb.active)
#     out_path = Path(out_path)

#     # ✅ duyệt theo từng sản phẩm
#     for prod, mob_dict in matrices_by_mob.items():
#         ws = wb.create_sheet(str(prod)[:31])
#         base_row = 1
#         base_col = 1

#         # Kiểm tra có score level hay không
#         has_score = any(isinstance(v, dict) for v in mob_dict.values())

#         if not has_score:
#             # --- Chỉ có MOB-level ---
#             all_mobs = sorted(mob_dict.keys())
#             for mob in all_mobs:
#                 mat = mob_dict[mob]
#                 header = f"{prod}_MOB{mob}"
#                 ws.cell(row=base_row, column=base_col, value=header).font = Font(bold=True)
#                 ws.cell(row=base_row, column=base_col).alignment = Alignment(horizontal="center")

#                 df_mat = mat.round(3).reset_index()
#                 for i, r in enumerate(dataframe_to_rows(df_mat, index=False, header=True)):
#                     for j, val in enumerate(r, start=0):
#                         ws.cell(row=base_row + 1 + i, column=base_col + j, value=val)
#                 # Dọc hay ngang
#                 if transpose:
#                     base_row += mat.shape[0] + 5
#                 else:
#                     base_col += mat.shape[1] + 3

#         else:
#             # --- Có 3 tầng: MOB → Score → matrix ---
#             all_scores = sorted({s for mob in mob_dict.values() for s in mob.keys()})
#             all_mobs = sorted(mob_dict.keys())

#             for score in all_scores:
#                 ws.cell(row=base_row, column=base_col, value=f"Score: {score}").font = Font(bold=True)
#                 ws.cell(row=base_row, column=base_col).alignment = Alignment(horizontal="left")

#                 # reset cursor theo chiều transpose
#                 row_cursor = base_row + 1
#                 col_cursor = base_col

#                 for mob in all_mobs:
#                     if mob not in mob_dict:
#                         continue
#                     mat_dict = mob_dict[mob]
#                     if score not in mat_dict:
#                         continue

#                     mat = mat_dict[score]
#                     header = f"{prod}_{score}_MOB{mob}"
#                     ws.cell(row=row_cursor, column=col_cursor, value=header).font = Font(bold=True)
#                     ws.cell(row=row_cursor, column=col_cursor).alignment = Alignment(horizontal="center")

#                     df_mat = mat.round(3).reset_index()
#                     for i, r in enumerate(dataframe_to_rows(df_mat, index=False, header=True)):
#                         for j, val in enumerate(r, start=0):
#                             ws.cell(row=row_cursor + 1 + i, column=col_cursor + j, value=val)

#                     # di chuyển con trỏ theo hướng hiển thị
#                     if transpose:
#                         row_cursor += mat.shape[0] + 5
#                     else:
#                         col_cursor += mat.shape[1] + 3

#                 # xong 1 score, xuống dòng thêm để tách block
#                 if transpose:
#                     base_row = row_cursor + 2
#                 else:
#                     base_row += mat.shape[0] + 6
#                     base_col = 1

#     wb.save(out_path)
#     print(f"✅ Xuất ma trận Markov hoàn tất (transpose={transpose}): {out_path}")


In [ ]:
export_transition_excel(
    matrices_by_mob,
    out_path=OUT_ROOT / "Transition_ByProduct_Score_MOB.xlsx",
    transpose=True   # 👉 bật dọc (MOB theo hàng)
)


In [ ]:
# 1) Kiểu và số phần tử cấp 1
print("type:", type(matrices_by_mob))
print("len(level-1):", len(matrices_by_mob))
print("level-1 keys (sample):", list(matrices_by_mob.keys())[:10])


In [ ]:
# 2) Duyệt 1-2 phần tử đầu tiên: xem cấp 2 là gì (MOB hay gì đó)
for i, (k1, v1) in enumerate(matrices_by_mob.items()):
    print(f"\n[#{i}] level-1 key = {k1} | type(v1) = {type(v1)} | len(v1) = {len(v1)}")
    if isinstance(v1, dict):
        sub_keys = list(v1.keys())[:10]
        print("  level-2 keys (sample):", sub_keys)
        # thử nhìn 1 phần tử cấp 2
        k2 = sub_keys[0]
        v2 = v1[k2]
        print(f"  type(level-2 value) = {type(v2)}")
        if isinstance(v2, dict):
            sub_keys2 = list(v2.keys())[:10]
            print("    level-3 keys (sample):", sub_keys2)
            # thử nhìn 1 phần tử cấp 3
            k3 = sub_keys2[0]
            v3 = v2[k3]
            print(f"    type(level-3 value) = {type(v3)} | shape={getattr(v3, 'shape', None)}")
        else:
            print(f"  shape(level-2 matrix) = {getattr(v2, 'shape', None)}")
    if i >= 1:
        break


In [ ]:
# 3) Thống kê nhanh xem cấp 2/3 có phải dict (MOB/Score) hay DataFrame (ma trận)
from collections import Counter
lvl2_types = Counter(type(v) for v in matrices_by_mob.values())
print("level-2 value types:", lvl2_types)

# Nếu cấp 2 là dict, check sâu thêm 1 tầng
lvl3_types = Counter()
for v1 in matrices_by_mob.values():
    if isinstance(v1, dict):
        for v2 in v1.values():
            lvl3_types[type(v2)] += 1
print("level-3 value types:", lvl3_types)


In [ ]:
# 3) Thống kê nhanh xem cấp 2/3 có phải dict (MOB/Score) hay DataFrame (ma trận)
from collections import Counter
lvl2_types = Counter(type(v) for v in matrices_by_mob.values())
print("level-2 value types:", lvl2_types)

# Nếu cấp 2 là dict, check sâu thêm 1 tầng
lvl3_types = Counter()
for v1 in matrices_by_mob.values():
    if isinstance(v1, dict):
        for v2 in v1.values():
            lvl3_types[type(v2)] += 1
print("level-3 value types:", lvl3_types)


In [ ]:
# 4) In 1-2 ma trận cụ thể (nếu có)
try:
    first_prod = next(iter(matrices_by_mob))
    second_level = matrices_by_mob[first_prod]
    if isinstance(second_level, dict):
        first_k2 = next(iter(second_level))
        maybe_df = second_level[first_k2]
        if hasattr(maybe_df, "shape"):
            print(f"\nMatrix sample at [{first_prod}][{first_k2}] shape={maybe_df.shape}")
            display(maybe_df.head())
        elif isinstance(maybe_df, dict):
            first_k3 = next(iter(maybe_df))
            mat = maybe_df[first_k3]
            print(f"\nMatrix sample at [{first_prod}][{first_k2}][{first_k3}] shape={mat.shape}")
            display(mat.head())
    else:
        print("level-2 is not a dict; unexpected structure.")
except StopIteration:
    print("matrices_by_mob is empty.")


In [ ]:
# 5) Nếu bạn kỳ vọng: dict[product][mob][score] = DataFrame
#    Kiểm tra “hình” đúng như vậy chưa
def check_expected_shape(mb):
    ok = True
    for prod, mob_dict in mb.items():
        if not isinstance(mob_dict, dict):
            print(f"❌ prod={prod} không phải dict → {type(mob_dict)}"); ok=False; continue
        for mob, score_or_mat in mob_dict.items():
            if not isinstance(score_or_mat, dict):
                print(f"❌ mob-level là {type(score_or_mat)} (mong đợi dict score) | prod={prod}, mob={mob}")
                ok=False; continue
            for score, mat in score_or_mat.items():
                if not hasattr(mat, 'shape'):
                    print(f"❌ value tại score={score} không phải DataFrame | type={type(mat)} | prod={prod}, mob={mob}")
                    ok=False
    print("✅ cấu trúc đúng kỳ vọng." if ok else "⚠️ cấu trúc KHÔNG đúng kỳ vọng.")

check_expected_shape(matrices_by_mob)


In [ ]:
mob_df = df[(df["PRODUCT_TYPE"] == "SALPIL") & (df["SCORE"] == "A")]
mob_df.groupby("AGREEMENT_ID")["MOB"].agg(["min", "max", "count"]).head(10)


In [ ]:
list(matrices_by_mob.keys())[:5]


In [ ]:
from src.rollrate.export_excel import export_transition_excel

export_transition_excel(
    matrices_by_mob,
    out_path=r"D:\Python_code\RR_Model\outputs\RRM_transition_full.xlsx"
)


In [ ]:
mob_df = df[(df["PRODUCT_TYPE"] == "SALPIL") & (df["RISK_SCORE"] == "A")]
mob_df.groupby("AGREEMENT_ID")["MOB"].agg(["min", "max", "count"]).head(10)


In [ ]:
pip install pyarrow

In [ ]:
import pandas as pd

# Giả sử DataFrame của bạn tên là df
df.to_parquet("ETB.parquet", compression="snappy", index=False)


In [ ]:
print("\n==============================================")
print("STEP 6 – FULL DEBUG: RAW → PAIRS → WEIGHTS → TRANSITION")
print("==============================================\n")

debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

loan_col  = CFG["loan"]
mob_col   = "mob_t"
state_col = "state_t"
cutoff    = "CUTOFF_DATE"
ead_col   = "ead_t"


# ------------------------------------------------------------
# 1) RAW DATA
# ------------------------------------------------------------
raw_slice = df[
    (df["PRODUCT_TYPE"] == debug_prod) &
    (df["RISK_SCORE"] == debug_score) &
    (df[CFG["mob"]].astype(int) == debug_mob)
].copy()

print(f"🔍 RAW DATA — PRODUCT={debug_prod}, SCORE={debug_score}, MOB={debug_mob}")
print(f"Số dòng raw   = {len(raw_slice):,}")
display(raw_slice.head())

print("\n📊 RAW State distribution:")
display(raw_slice[CFG["state"]].value_counts())


# ------------------------------------------------------------
# 2) PAIRS (t → t+1)
# ------------------------------------------------------------
pairs_slice = pairs_all[
    (pairs_all["product_t"] == debug_prod) &
    (pairs_all["score_t"] == debug_score) &
    (pairs_all["mob_t"] == debug_mob)
].copy()

print("\n🔍 PAIRS (t → t+1) của nhóm này")
print(f"Số pairs = {len(pairs_slice):,}")
display(pairs_slice.head())

print("\n📊 Pair count (state_t → state_t1):")
display(
    pairs_slice.groupby(["state_t", "state_t1"])
               .size()
               .unstack(fill_value=0)
)


# ------------------------------------------------------------
# 3) Xác định loans bị mất tháng kế tiếp
# ------------------------------------------------------------
loans_raw  = set(raw_slice[loan_col])
loans_pair = set(pairs_slice[loan_col])

missing = loans_raw - loans_pair
print(f"\n🚨 Loans thiếu mob_t+1 (không tạo được cặp): {len(missing)}")
display(list(missing)[:30])


# ------------------------------------------------------------
# 4) FULL LOAN TIMELINE
# ------------------------------------------------------------
print("\n📘 FULL LOAN TIMELINE — kiểm tra chuỗi MOB từng hợp đồng")
timeline = df[df[loan_col].isin(loans_raw)] \
    .sort_values([loan_col, CFG["mob"]])
display(timeline.head(50))


# ------------------------------------------------------------
# 5) TÍNH WEIGHT WHA CHÍNH XÁC (theo ROLL_WINDOW & WEIGHT_METHOD)
# ------------------------------------------------------------
from src.config import WEIGHT_METHOD, ROLL_WINDOW

ps = pairs_slice.copy()
ps[cutoff] = pd.to_datetime(ps["cutoff_t"], errors="coerce")

# Convert cutoff_date → monthly index
month_idx = ps[cutoff].dt.to_period("M").astype(int)
max_idx   = month_idx.max()

age = (max_idx - month_idx).clip(0, ROLL_WINDOW)

if WEIGHT_METHOD == "exp":
    lam = CFG.get("DECAY_LAMBDA", 0.9)
    ps["time_weight"] = (lam ** age).astype(float)
elif WEIGHT_METHOD == "linear":
    ps["time_weight"] = (1 - age / ROLL_WINDOW).clip(0)
else:
    ps["time_weight"] = 1.0

# final weight = EAD × time_weight
ps["weight"] = ps[ead_col] * ps["time_weight"]

print("\n🔍 WEIGHTED PAIRS (WHA × EAD)")
display(ps.head(20))

print("\n📊 Total Weight by state_t → state_t1:")
display(
    ps.groupby(["state_t", "state_t1"])["weight"]
      .sum()
      .unstack(fill_value=0)
)


# ------------------------------------------------------------
# 6) MANUAL TRANSITION MATRIX (INPUT CHUẨN)
# ------------------------------------------------------------
manual_counts = (
    ps.groupby(["state_t", "state_t1"])["weight"]
      .sum()
      .unstack(fill_value=0)
      .reindex(index=BUCKETS_CANON, columns=BUCKETS_CANON, fill_value=0)
)

manual_P = manual_counts.div(manual_counts.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

print("\n📊 MANUAL WEIGHTED TRANSITION MATRIX (đây là P thật trước khi smoothing):")
display(manual_P.round(4))


In [ ]:
# ============================================================
# 0) Tạo pairs_all từ raw
# ============================================================

print("=== RUN make_pairs() ===")
pairs_all = make_pairs(df)    # MUST RUN BEFORE ANY DEBUG
print(f"✔ Tổng số pairs tạo ra: {len(pairs_all):,}")

# ============================================================
# 1) Chọn nhóm để debug
# ============================================================

debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

loan_col   = CFG["loan"]
mob_col    = "mob_t"
state_col  = CFG["state"]
ead_raw_col = CFG.get("ead")       # EAD raw trong df
cutoff_raw_col = CFG.get("cutoff") # tên cutoff trong df raw, ví dụ "CUTOFF_DATE"

# ============================================================
# 2) Lọc đúng nhóm (product, score, mob)
# ============================================================

pairs_slice = pairs_all[
    (pairs_all["product_t"] == debug_prod) &
    (pairs_all["score_t"] == debug_score) &
    (pairs_all["mob_t"] == debug_mob)
].copy()

print("\n🔍 PAIRS (t → t+1) của nhóm này")
print(f"Số pairs = {len(pairs_slice):,}")
display(pairs_slice.head())


In [ ]:
# ============================================================
# JOIN cutoff_date + EAD_RAW từ raw df
# ============================================================

# Chuẩn bị df raw key
df_key = df.set_index([loan_col, CFG["mob"]])

# cutoff_date (nếu có)
if cutoff_raw_col in df.columns:
    pairs_slice["cutoff_t"] = pairs_slice.apply(
        lambda r: df_key.loc[(r[loan_col], r["mob_t"]), cutoff_raw_col]
        if (r[loan_col], r["mob_t"]) in df_key.index else pd.NaT,
        axis=1
    )
else:
    pairs_slice["cutoff_t"] = pd.NaT

# EAD raw (nếu có)
if ead_raw_col in df.columns:
    pairs_slice["EAD_RAW"] = pairs_slice.apply(
        lambda r: df_key.loc[(r[loan_col], r["mob_t"]), ead_raw_col]
        if (r[loan_col], r["mob_t"]) in df_key.index else None,
        axis=1
    )
else:
    pairs_slice["EAD_RAW"] = None


In [ ]:
# ============================================================
# 4) TÍNH WEIGHT (WHA) – FIXED VERSION
# ============================================================

pairs_slice["cutoff_t"] = pd.to_datetime(pairs_slice["cutoff_t"], errors="coerce")

# convert Period safely → string → Int64
month_idx = (
    pairs_slice["cutoff_t"]
    .dt.to_period("M")
    .astype(str)
    .replace("NaT", pd.NA)
    .astype("Int64")
)

max_idx = month_idx.max()

age = (max_idx - month_idx).clip(lower=0, upper=CFG["ROLL_WINDOW"])

WM = CFG["WEIGHT_METHOD"]

if WM == "exp":
    lam = CFG.get("DECAY_LAMBDA", 0.9)
    pairs_slice["time_weight"] = (lam ** age).astype(float)

elif WM == "linear":
    pairs_slice["time_weight"] = (1 - age / CFG["ROLL_WINDOW"]).clip(lower=0)

else:
    pairs_slice["time_weight"] = 1.0

pairs_slice["weight"] = pairs_slice["ead_t"] * pairs_slice["time_weight"]


In [ ]:
cols_show = [
    loan_col, "mob_t", "mob_t1",
    "state_t", "state_t1",
    "EAD_RAW", "ead_t",
    "cutoff_t", "time_weight", "weight"
]

print("\n🔍 DỮ LIỆU SAU make_pairs + JOIN cutoff + WHA")
display(pairs_slice[cols_show].head(50))


In [ ]:
# 1) Tạo pairs toàn bộ (như trong compute_transition_by_mob)
pairs_all = make_pairs(df)

# 2) Lấy đúng slice mà transition dùng
debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

pairs_slice = pairs_all[
    (pairs_all["product_t"] == debug_prod)
    & (pairs_all["score_t"] == debug_score)
    & (pairs_all["mob_t"] == debug_mob)
].copy()

print("🔍 Pairs dùng để tính transition (đã filter)")
print(f"Số pairs = {len(pairs_slice):,}")
display(pairs_slice)


In [ ]:
# Nếu bạn dùng weight WHA (time-decay)
pairs_slice["weight"] = pairs_slice["ead_t"] * pairs_slice["time_weight"]

# Nếu không có time_weight (chưa tính)
pairs_slice["weight"] = pairs_slice["ead_t"]


In [ ]:
def debug_transition_group(df, debug_prod, debug_score, debug_mob):
    """
    Debug chi tiết 1 nhóm Product × Score × MOB:
    - Raw data
    - Pairs sau make_pairs với full weight: ead_raw, time_weight, ead_t
    - Manual aggregation
    - P_model
    - Export pairs_slice → Excel
    """

    print("\n================= DEBUG TRANSITION GROUP =================")
    print(f"→ product = {debug_prod}, score = {debug_score}, mob = {debug_mob}")

    loan   = CFG["loan"]
    mob    = CFG["mob"]
    state  = CFG["state"]
    cutoff = CFG["cutoff"]
    eadcol = CFG.get("ead")

    # ======================================================
    # 0) RAW trước make_pairs
    # ======================================================
    raw_slice = df[
        (df["PRODUCT_TYPE"] == debug_prod) &
        (df["RISK_SCORE"] == debug_score) &
        (df[mob] == debug_mob)
    ].copy()

    print("\n=========== RAW DATA ===========")
    if raw_slice.empty:
        print("❌ Không có raw data.")
    else:
        display(raw_slice.head(50))

    # ======================================================
    # 1) Pairs toàn dataset
    # ======================================================
    pairs_all = make_pairs(df)

    # ======================================================
    # 2) Pairs nhóm cần debug
    # ======================================================
    print("\n=========== PAIRS (t→t+1) OF GROUP ===========")
    pairs_slice = pairs_all[
        (pairs_all["product_t"] == debug_prod) &
        (pairs_all["score_t"] == debug_score) &
        (pairs_all["mob_t"] == debug_mob)
    ].copy()

    print(f"Số pairs nhóm này = {len(pairs_slice):,}")

    if pairs_slice.empty:
        print("❌ Không có pairs cho nhóm này.")
    else:
        show_cols = [
            loan, "mob_t", "mob_t1",
            "state_t", "state_t1",
            "ead_raw", "time_weight", "ead_t"
        ]
        display(pairs_slice[show_cols].head(50))

        # =============================
        #  🔽 Xuất ra Excel
        # =============================
        out_path = f"debug_pairs_{debug_prod}_{debug_score}_MOB{debug_mob}.xlsx"
        pairs_slice.to_excel(out_path, index=False)
        print(f"📄 Đã xuất file chi tiết pairs → {out_path}")

    # ======================================================
    # 3) Manual aggregate
    # ======================================================
    if not pairs_slice.empty:
        print("\n=========== MANUAL AGG (state_t → state_t1) ===========")
        manual_ct = (
            pairs_slice
            .groupby(["state_t", "state_t1"])["ead_t"]
            .sum()
            .unstack(fill_value=0)
            .reindex(index=STATE_SPACE, columns=STATE_SPACE, fill_value=0)
        )
        display(manual_ct)

        print("\n=========== MANUAL NORMALIZED MATRIX ===========")
        manual_p = manual_ct.div(
            manual_ct.sum(axis=1).replace(0, np.nan), axis=0
        ).fillna(0)
        display(manual_p)
    else:
        manual_ct = manual_p = None

    # ======================================================
    # 4) Model matrix
    # ======================================================
    print("\n=========== MODEL MATRIX ===========")
    matrices_by_mob, parent_fallback = compute_transition_by_mob(df)

    try:
        obj = matrices_by_mob[debug_prod][debug_mob][debug_score]
        P_model = obj["P"]
        print("is_fallback:", obj["is_fallback"])
        print("reason:", obj["reason"])
        display(P_model)
    except Exception as e:
        print("❌ Không lấy được model matrix:", e)
        P_model = None

    return {
        "raw_slice": raw_slice,
        "pairs_slice": pairs_slice,
        "manual_counts": manual_ct,
        "manual_matrix": manual_p,
        "P_model": P_model,
    }


In [ ]:
pairs_slice.to_excel('check_details.xlsx')

In [ ]:
debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

debug_out = debug_transition_group(df, debug_prod, debug_score, debug_mob)


In [ ]:
debug_transition_group(df, "SALPIL", "A", 0)


In [ ]:
def debug_transition_group(df, debug_prod, debug_score, debug_mob):
    """
    Debug đầy đủ:
      - Raw data của nhóm theo ROLL_WINDOW
      - Weight (ead_raw, time_weight, ead_t)
      - Pairs (t → t+1)
      - Manual matrix
      - Matrix từ code
    """

    loan   = CFG["loan"]
    mob    = CFG["mob"]
    state  = CFG["state"]
    cutoff = CFG["cutoff"]

    print("\n" + "="*80)
    print(f"DEBUG GROUP ⇒ product={debug_prod}, score={debug_score}, MOB={debug_mob}")
    print("="*80)

    # --------------------------------------------------------------------------------------
    # 1) RUN make_pairs ONCE (FULL DATA) – This now returns: ead_raw, time_weight, ead_t
    # --------------------------------------------------------------------------------------
    pairs_all = make_pairs(df)

    print(f"\n🔍 Tổng số pairs toàn dataset = {len(pairs_all):,}")

    # --------------------------------------------------------------------------------------
    # 2) Lấy raw của nhóm này trong pairs
    # --------------------------------------------------------------------------------------
    pairs_slice = pairs_all[
        (pairs_all["product_t"] == debug_prod) &
        (pairs_all["score_t"] == debug_score) &
        (pairs_all["mob_t"] == debug_mob)
    ].copy()

    print(f"\n🔍 PAIRS (t → t+1) CỦA NHÓM NÀY = {len(pairs_slice):,}")

    if pairs_slice.empty:
        print("❌ Nhóm này không có bất kỳ pair nào.")
        return None

    # --------------------------------------------------------------------------------------
    # 3) SHOW DETAILED PAIRS
    # --------------------------------------------------------------------------------------
    show_cols = [
        loan,
        "mob_t", "mob_t1",
        "state_t", "state_t1",
        "ead_raw",
        "time_weight",
        "ead_t",
        "product_t", "score_t",
    ]

    print("\n=========== DỮ LIỆU PAIRS CHI TIẾT ===========")
    display(pairs_slice[show_cols].head(200))

    # --------------------------------------------------------------------------------------
    # 4) MANUAL AGGREGATION
    # --------------------------------------------------------------------------------------
    print("\n=========== MANUAL AGG (state_t → state_t1) ===========")

    manual_ct = (
        pairs_slice
        .groupby(["state_t", "state_t1"])["ead_t"]
        .sum()
        .unstack(fill_value=0)
        .reindex(index=BUCKETS_CANON + list(ABSORBING_BASE),
                 columns=BUCKETS_CANON + list(ABSORBING_BASE),
                 fill_value=0)
    )

    display(manual_ct)

    print("\n=========== MANUAL NORMALIZED MATRIX ===========")
    manual_P = manual_ct.div(manual_ct.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    display(manual_P.round(4))

    # --------------------------------------------------------------------------------------
    # 5) MATRIX FROM CODE (compute_transition_from_pairs)
    # --------------------------------------------------------------------------------------
    print("\n=========== MATRIX FROM CODE ===========")
    code_P = compute_transition_from_pairs(
        pairs_slice,
        value_col="ead_t",
        parent_P=None,
        zero_row_policy="parent",
        alpha_smooth=0.0,
    )
    display(code_P.round(4))

    # --------------------------------------------------------------------------------------
    # 6) DIFFERENCE manual vs code
    # --------------------------------------------------------------------------------------
    print("\n=========== DIFFERENCE (manual - code) ===========")
    diff = (manual_P - code_P).round(4)
    display(diff)

    print("\n=== DONE DEBUG ===")

    return {
        "pairs": pairs_slice,
        "manual_counts": manual_ct,
        "manual_P": manual_P,
        "code_P": code_P,
        "diff": diff
    }


In [ ]:
debug_prod  = "SALPIL"
debug_score = "A"
debug_mob   = 0

debug_out = debug_transition_group(df, debug_prod, debug_score, debug_mob)


In [ ]:
if debug_out:
    debug_out["pairs"].to_excel("debug_pairs_detail2.xlsx", index=False)
    print("✅ Saved → debug_pairs_detail2.xlsx")


In [ ]:
from src.rollrate.transition import compute_transition_by_mob

print("\n=== Tính transition matrices theo Product × MOB × Score ===")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)


In [ ]:
from src.rollrate.forecast import forecast_dynamic_mob

In [ ]:
cutoff_col = CFG["cutoff"]
mob_col    = CFG["mob"]

# lấy cutoff mới nhất
latest_cutoff = df[cutoff_col].max()

df_group = df[
    (df["PRODUCT_TYPE"] == product) &
    (df["RISK_SCORE"] == score) &
    (df[cutoff_col] == latest_cutoff)
].copy()

print(f"Số hợp đồng đầu vào forecast: {len(df_group):,}")


In [ ]:
forecast_df = forecast_dynamic_mob(
    df_group=df_group,
    matrices_by_mob=matrices_by_mob,
    product=product,
    score=score,
    max_mob_forecast=max_mob,   # số ma trận MOB bạn có
    value_col="PRINCIPLE_OUTSTANDING"
)


In [ ]:
forecast_df.round(4)


In [ ]:


res = forecast_one_group(
    df,
    matrices_by_mob,
    parent_fallback,
    product="SALPIL",
    score="A",
    months=12
)

display(res)


In [ ]:
def get_initial_distribution(df_group, state_col="STATE", value_col="EAD"):
    dist = (
        df_group.groupby(state_col)[value_col]
        .sum()
        .reindex(STATE_SPACE, fill_value=0)
    )
    total = dist.sum()
    if total > 0:
        dist = dist / total
    return dist


In [ ]:
product = "SALPIL"
score = "A"

# lấy group tại cutoff gần nhất
cutoff_col = CFG["cutoff"]
latest_cutoff = df[cutoff_col].max()

df_group = df[
    (df["PRODUCT_TYPE"] == product) &
    (df["RISK_SCORE"] == score) &
    (df[cutoff_col] == latest_cutoff)
].copy()

initial_mob = df_group["MOB"].max()         # hoặc min, tùy logic bạn dùng
initial_dist = get_initial_distribution(df_group)

max_mob = max(matrices_by_mob[product].keys())

forecast_res = forecast_segment(
    matrices_by_mob,
    product,
    score,
    initial_mob,
    initial_dist,
    max_mob=max_mob
)


In [ ]:
df_forecast = pd.DataFrame(forecast_res).T
df_forecast.index.name = "MOB"
df_forecast


In [ ]:
from src.rollrate.forecast import forecast_full_segment

In [ ]:
from src.rollrate.forecast import forecast_full_segment

print("OK, forecast_full_segment loaded!")


In [ ]:
import inspect


print(inspect.getsource(fc))


In [ ]:
!python -m py_compile src/rollrate/forecast.py


In [ ]:
import os
print(os.getcwd())


In [ ]:
!find . -maxdepth 3 -type d


In [ ]:
!dir /s /b forecast.py


In [ ]:
!dir C:\Users\User\RR_model\src\rollrate\


In [ ]:
fc = forecast_monthly(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    product="CASH",
    score="S1",
    max_mob=36,             # forecast đến MOB 36
    enable_macro=False
)

print(fc)
from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))
from src.rollrate.forecast import forecast_monthly,forecast_to_dataframe, forecast_vintage

# 🎯 Forecast 29 MOB tới cho Product="CASH", Score="S1"
fc_29 = forecast_monthly(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    product="SALPIL",
    score="A",
    max_mob=29,
    enable_macro=False
)

# Convert sang DataFrame để xem dễ hơn
df_fc_29 = forecast_to_dataframe(fc_29)
print(df_fc_29)


In [19]:
from src.rollrate.forecast import (

    forecast_to_dataframe,
    forecast_all_vintages
)


In [17]:
from src.rollrate.forecast import forecast_monthly, forecast_to_dataframe, forecast_all_vintages

fc_29 = forecast_monthly(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    product="SALPIL",
    score="A",
    max_mob=29,
    enable_macro=False,
)


ImportError: cannot import name 'forecast_monthly' from 'src.rollrate.forecast' (C:\Users\User\RR_model\src\rollrate\forecast.py)

In [ ]:
all_vintages = df["DISBURSAL_DATE"].dropna().unique()

results = {}

for v in all_vintages:
    results[v] = forecast_vintage(
        df_raw=df,
        matrices_by_mob=matrices_by_mob,
        product="SALPIL",
        score="A",
        vintage_date=v,
        max_mob=29
    )


In [21]:
from src.rollrate.forecast import forecast_all_vintages, forecast_to_dataframe

results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)


⚠️ Skip (SALPIL, A, v=2023-01-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-02-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-03-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-04-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-05-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-06-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-07-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-08-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-09-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-10-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-11-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2023-12-01 00:00:00): name 'STATE_SPACE' is not defined
⚠️ Skip (SALPIL, A, v=2024-01-01 00:00:00): name 'STATE_SPACE' i

In [15]:
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))